In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY,
)

## Runnable Passthrough

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("yo")

'yo'

## Runnable Lambda

In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [7]:
chain.invoke("harsha")

'harshaovich'

## Runnable Parallel

In [8]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname),
    }
)

In [9]:
chain.invoke("Harsha")

{'operation_a': 'Harsha', 'operation_b': 'Harshaovich'}

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

output_parser = StrOutputParser()

In [11]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [12]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "topic": RunnableLambda(russian_lastname),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | llm | output_parser

In [15]:
chain.invoke({"name": "harsha"})

'Harsha-ovich walks into a café.\n\nThe barista looks at his name on the order and says, "Harsha-ovich? Wow, that\'s a mouthful! So, are you having a chai latte, or a double espresso, comrade?"\n\nHarsha-ovich smiles. "Just a plain black coffee, please. I\'m complicated enough already!"'